In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import urllib.request, urllib.parse, urllib.error
# import
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

In [2]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [3]:
path80 = []
for i in range(1, 1001):
    path = 'https://www.seek.com.au/data-jobs/in-All-Australia?page=' + str(i) + '&salaryrange=0-80000&salarytype=annual'
    path80.append(path)

In [4]:
urls80 = []
dupe_count = 0
for path in log_progress(path80):
    # load the page
    page = requests.get(path)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # grab title url
    titles = soup.find_all('a', {'data-automation': 'jobTitle'}) 
    for title in titles:
        url = 'https://www.seek.com.au' + title.get('href')
        if url not in urls80:
            urls80.append(url)
        else:
            dupe_count+=1

print('duplicates', dupe_count)


duplicates 194


In [5]:
urls80_df = pd.DataFrame(urls80, columns = ['urls'])

In [6]:
urls80_df.to_csv('seek_urls_80k.csv'

### Scrape all 80k jobs

In [7]:
urls80_df = pd.read_csv('seek_urls_80k.csv')

In [8]:
urls80 = urls80_df.urls.values.tolist()

In [9]:
title_list = []
company_list = []
date_list = []
location_list = []
work_type_list = []
job_type_list = []
description_list = []
url_list = []

for url in log_progress(urls80):
    detailed_path = url
    page = requests.get(detailed_path)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    ## URL
    url_list.append(url)
    
    ## TITLES
    title_object = soup.find('span', {'data-automation': 'job-detail-title'})
    title = None
    try:
        title = title_object.text
    except:
        pass
    
    title_list.append(title)
    
    ## COMPANY
    company = None

    try:
        company_object = soup.find('span', {'data-automation': 'advertiser-name'})
        company = company_object.text

    except:
        pass
    
    company_list.append(company)
    
    
    ## DATE
    date_object = soup.find('dd', {'data-automation': 'job-detail-date'})
    date = None

    try:
        date = date_object.text
    except:
        pass
    
    date_list.append(date)
    
    ## LOCATION
    location = None
    try:
        info_header = soup.find('section', {'aria-labelledby': 'jobInfoHeader'})
        location_object = info_header.find('strong', {'class': 'lwHBT6d'})
        location= location_object.text
    except:
        pass
    
    location_list.append(location)
    
    ## WORK TYPE
    work_type = None

    try:
        work_type_object = soup.find('dd',{'data-automation': 'job-detail-work-type'})
        work_type = work_type_object.text
    except:
        pass
    
    work_type_list.append(work_type)
    
    ## JOB TYPE
    job_type = None

    try:
        info_header = soup.find('section', {'aria-labelledby': 'jobInfoHeader'})
        job_type = info_header.find_all('dd')[-1].find('strong', {'class': 'lwHBT6d'}).text
    except:
        pass

    job_type_list.append(job_type)
    
    ## DESCRIPTION
    description = None

    try:
        description_object = soup.find('div', {'data-automation': 'jobDescription'})
        for p in description_object.find_all('p'):
            if description == None:
                description = p.text
            else: 
                description = description + ' ' + p.text
    except:
        pass


    try:
        description_object = soup.find('div', {'data-automation': 'jobDescription'})
        for li in description_object.find_all('li'):
            if description == None:
                description = li.text
            else: 
                description = description + ' ' + li.textb
    except:
        pass
    
    description_list.append(description)
    

seek80k_jobs_df = pd.DataFrame({'url': url_list,
                               'job_title': title_list,
                               'company': company_list,
                               'date': date_list,
                               'location': location_list,
                               'employment_type': work_type_list,
                               'field': job_type_list,
                               'description': description_list})

seek80k_jobs_df.to_csv('seek80k_jobs.csv')


In [10]:
seek80k_jobs_df.head()


,url,job_title,company,date,location,employment_type,field,description
0,https://www.seek.com.au/job/37428435?type=prom...,Tender / Bid Leader - Data Collection & Road A...,Australian Road Research Board,10 Oct 2018,Brisbane,Full Time,Engineering,"About us:Established in 1960, the Australian R..."
1,https://www.seek.com.au/job/37512965?type=prom...,Commercial Analyst,None,19 Oct 2018,Cairns & Far North,Full Time,Accounting,Coral Expeditions is Australia's Pioneering Ex...
2,https://www.seek.com.au/job/37523713?type=stan...,Data Analyst,Youfoodz Pty Ltd,22 Oct 2018,Brisbane,Full Time,Marketing & Communications,You've heard about us…You've watched as we hav...
3,https://www.seek.com.au/job/37523959?type=stan...,Data Analyst,Polyglot Group,22 Oct 2018,Melbourne,Full Time,"Mining, Resources & Energy",Our client is an Australian company providing ...
4,https://www.seek.com.au/job/37522935?type=stan...,Data Administrator,None,22 Oct 2018,ACT,Part Time,Education & Training,Classification: ANU Officer 4 (IT)Salary packa...


## 100k+ jobs

In [11]:
path100 = []
for i in range(1, 1001):
    path = 'https://www.seek.com.au/data-jobs/in-Australia?page=' + str(i) + '&salaryrange=100000-999999&salarytype=annual'
    path100.append(path)
    
    

In [12]:
urls100 = []
dupe_count = 0
for path in log_progress(path100):
    # load the page
    page = requests.get(path)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # grab title url
    titles = soup.find_all('a', {'data-automation': 'jobTitle'}) 
    for title in titles:
        url = 'https://www.seek.com.au' + title.get('href')
        if url not in urls80 and url not in urls100:
            urls100.append(url)
        else:
            dupe_count+=1

print('duplicates', dupe_count)


duplicates 197


In [13]:
urls100_df = pd.DataFrame(urls100, columns = ['urls'])

In [14]:
urls100_df.to_csv('seek_urls_100k.csv')

### Scrape all 100k jobs

In [15]:
urls100_df = pd.read_csv('seek_urls_100k.csv')

In [16]:
urls100 = urls100_df.urls.values.tolist()

In [17]:
title_list = []
company_list = []
date_list = []
location_list = []
work_type_list = []
job_type_list = []
description_list = []
url_list = []

for url in log_progress(urls100):
    detailed_path = url
    page = requests.get(detailed_path)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    ## URL
    url_list.append(url)
    
    ## TITLES
    title_object = soup.find('span', {'data-automation': 'job-detail-title'})
    title = None
    try:
        title = title_object.text
    except:
        pass
    
    title_list.append(title)
    
    ## COMPANY
    company = None

    try:
        company_object = soup.find('span', {'data-automation': 'advertiser-name'})
        company = company_object.text

    except:
        pass
    
    company_list.append(company)
    
    
    ## DATE
    date_object = soup.find('dd', {'data-automation': 'job-detail-date'})
    date = None

    try:
        date = date_object.text
    except:
        pass
    
    date_list.append(date)
    
    ## LOCATION
    location = None
    try:
        info_header = soup.find('section', {'aria-labelledby': 'jobInfoHeader'})
        location_object = info_header.find('strong', {'class': 'lwHBT6d'})
        location= location_object.text
    except:
        pass
    
    location_list.append(location)
    
    ## WORK TYPE
    work_type = None

    try:
        work_type_object = soup.find('dd',{'data-automation': 'job-detail-work-type'})
        work_type = work_type_object.text
    except:
        pass
    
    work_type_list.append(work_type)
    
    ## JOB TYPE
    job_type = None

    try:
        info_header = soup.find('section', {'aria-labelledby': 'jobInfoHeader'})
        job_type = info_header.find_all('dd')[-1].find('strong', {'class': 'lwHBT6d'}).text
    except:
        pass

    job_type_list.append(job_type)
    
    ## DESCRIPTION
    description = None

    try:
        description_object = soup.find('div', {'data-automation': 'jobDescription'})
        for p in description_object.find_all('p'):
            if description == None:
                description = p.text
            else: 
                description = description + ' ' + p.text
    except:
        pass


    try:
        description_object = soup.find('div', {'data-automation': 'jobDescription'})
        for li in description_object.find_all('li'):
            if description == None:
                description = li.text
            else: 
                description = description + ' ' + li.text
    except:
        pass
    
    description_list.append(description)
    
    
seek100k_jobs_df = pd.DataFrame({'url': url_list,
                               'job_title': title_list,
                               'company': company_list,
                               'date': date_list,
                               'location': location_list,
                               'employment_type': work_type_list,
                               'field': job_type_list,
                               'description': description_list})

seek100k_jobs_df.to_csv('seek100k_jobs.csv')

In [18]:
seek100k_jobs_df.field.isnull().sum()

0

### Try scraping detailed page

In [ ]:
detailed_path = 'https://www.seek.com.au/job/37457759?type=standard'
page = requests.get(detailed_path)
soup = BeautifulSoup(page.content, 'html.parser')





#### Title

In [ ]:
# TITLE
title_object = soup.find('span', {'data-automation': 'job-detail-title'})
title = ''
try:
    title = title_object.text
except:
    title = None

In [ ]:
title

#### Company type

In [ ]:
company = ''

try:
    company_object = soup.find('span', {'data-automation': 'advertiser-name'})
    company = company_object.text
    
except:
    company = None
    

In [ ]:
company

#### Date

In [ ]:
date_object = soup.find('dd', {'data-automation': 'job-detail-date'})
date = ''

try:
    date = date_object.text
except:
    date = None

In [ ]:
date

#### Location

In [ ]:
location = ''
try:
    info_header = soup.find('section', {'aria-labelledby': 'jobInfoHeader'})
    location_object = info_header.find('strong', {'class': 'lwHBT6d'})
    location= location_object.text
except:
    location = None

In [ ]:
location

#### Work type

In [ ]:
work_type = ''

try:
    work_type_object = soup.find('dd',{'data-automation': 'job-detail-work-type'})
    work_type = work_type_object.text
except:
    work_type = None

In [ ]:
work_type_object.text

#### Job type

In [ ]:
job_type = ''

try:
    info_header = soup.find('section', {'aria-labelledby': 'jobInfoHeader'})
    job_type = info_header.find_all('dd')[-1].find('strong', {'class': 'lwHBT6d'}).text
except:
    job_type = None

    

In [ ]:
job_type


#### Job description

In [ ]:
description = ''

description_object = soup.find('div', {'data-automation': 'jobDescription'})

In [ ]:
description = None

try:
    description_object = soup.find('div', {'data-automation': 'jobDescription'})
    for p in description_object.find_all('p'):
        if description == None:
            description = p.text
        else: 
            description = description + ' ' + p.text
except:
    pass
    
    
try:
    description_object = soup.find('div', {'data-automation': 'jobDescription'})
    for li in description_object.find_all('li'):
        if description == None:
            description = li.text
        else: 
            description = description + ' ' + li.text
except:
    pass

In [ ]:
description